# 1、ConversationTokenBufferMemory的使用

举例1：

In [2]:
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")

# 创建大模型实例
llm = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

In [2]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.messages import trim_messages

# ====================================================
# 1. 定义离线计数器 (防止网络卡顿的关键)
# ====================================================
def offline_counter(messages) -> int:
    """用字数模拟 Token 数，每 1 个字算 1 个 Token (粗略估算)"""
    total = 0
    for msg in messages:
        total += len(msg.content)
    return total

# ====================================================
# 2. 定义修剪器 (替代 ConversationTokenBufferMemory)
# ====================================================
# 对应旧版的 max_token_limit=10
trimmer = trim_messages(
    max_tokens=10,          # 限制大小
    strategy="last",        # 保留最新的
    token_counter=offline_counter, # 使用离线计数，秒运行
    include_system=True,
    allow_partial=False,    # 不切断单句话
)

# ====================================================
# 3. 操作流程
# ====================================================

# A. 存储消息 (使用 InMemoryChatMessageHistory)
history = InMemoryChatMessageHistory()
history.add_user_message("你好吗？")       # 4个字
history.add_ai_message("我很好，谢谢！")   # 6个字 (累计10)
history.add_user_message("今天天气如何？") # 6个字 (累计16)
history.add_ai_message("晴天，25度")       # 5个字 (累计21)

print(f"--- 原始内存 (共 {len(history.messages)} 条消息) ---")
print(history.messages)

# B. 获取修剪后的结果
# 这步操作通常在 Chain 内部自动做，这里手动演示效果
pruned_messages = trimmer.invoke(history.messages)

print(f"\n--- 优化后：修剪结果 (Max=10) ---")
# 因为最后一条 "晴天，25度" 占5个token，上一条 "今天天气..." 占6个
# 5+6=11 > 10，所以只能保留最后一条
print(pruned_messages)

--- 原始内存 (共 4 条消息) ---
[HumanMessage(content='你好吗？', additional_kwargs={}, response_metadata={}), AIMessage(content='我很好，谢谢！', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='今天天气如何？', additional_kwargs={}, response_metadata={}), AIMessage(content='晴天，25度', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

--- 优化后：修剪结果 (Max=10) ---
[AIMessage(content='晴天，25度', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]


举例2：

In [3]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.messages import trim_messages

# ====================================================
# 1. 定义离线计数器 (关键优化)
# ====================================================
# 为了防止 tiktoken 下载卡死，用“字数”粗略模拟 Token 数
# 在中文语境下，1个汉字通常对应 0.7~2 个 Token，这里简单按 1:1 计算用于演示
def offline_token_counter(messages) -> int:
    total = 0
    for msg in messages:
        total += len(msg.content)
    return total

# ====================================================
# 2. 定义修剪器 (替代 ConversationTokenBufferMemory)
# ====================================================
# 对应你原来的 max_token_limit=20
trimmer = trim_messages(
    max_tokens=20,          # 设置上限 (这里按字数算)
    strategy="last",        # 保留最新的消息
    token_counter=offline_token_counter, # 使用离线计数
    include_system=True,
    allow_partial=False,    # 不截断单条消息
)

# ====================================================
# 3. 操作流程
# ====================================================

# A. 存储消息 (使用 InMemoryChatMessageHistory)
# 这是一个全量的笔记本，记录所有历史，不会自动删除
history = InMemoryChatMessageHistory()

# Round 1
history.add_user_message("你好吗？")       # 4 字
history.add_ai_message("我很好，谢谢！")   # 6 字

# Round 2
history.add_user_message("今天天气如何？") # 6 字
history.add_ai_message("晴天，25度")       # 5 字

# B. 查看原始记忆
print(f"--- 原始全量记忆 (共 {len(history.messages)} 条) ---")
print(history.messages)

# C. 执行修剪 (模拟 load_memory_variables)
# 这一步通常在 Chain 内部做，这里手动展示效果
# 计算逻辑：
# 最后一条(5) + 倒数第二条(6) + 倒数第三条(6) = 17 < 20
# 如果再加倒数第四条(4) = 21 > 20，超标了！
# 所以预期结果：丢弃最早的 "你好吗？"，保留后三条 (或者根据 trim 策略保留成对)
pruned_messages = trimmer.invoke(history.messages)

print(f"\n--- 优化后：修剪结果 (Max=20) ---")
print(pruned_messages)

--- 原始全量记忆 (共 4 条) ---
[HumanMessage(content='你好吗？', additional_kwargs={}, response_metadata={}), AIMessage(content='我很好，谢谢！', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='今天天气如何？', additional_kwargs={}, response_metadata={}), AIMessage(content='晴天，25度', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

--- 优化后：修剪结果 (Max=20) ---
[AIMessage(content='我很好，谢谢！', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='今天天气如何？', additional_kwargs={}, response_metadata={}), AIMessage(content='晴天，25度', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]


# 2、SummaryMemory的使用

举例1：

如果实例化SummaryMemory前，没有历史消息，可以使用构造方法实例化

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser

# 1. 创建大模型
llm = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

# ==========================================================
# 2. 准备历史数据
# ==========================================================
# 先创建一个历史记录对象，把对话存进去
history = InMemoryChatMessageHistory()
history.add_user_message("你好")
history.add_ai_message("怎么了")
history.add_user_message("你是谁")
history.add_ai_message("我是AI助手小智")
history.add_user_message("初次对话，你能介绍一下你自己吗？")
history.add_ai_message("当然可以了。我是一个无所不能的小智。")

print(f"--- 原始对话条数: {len(history.messages)} ---")

# ==========================================================
# 3. 定义总结链 (替代 ConversationSummaryMemory 的黑盒逻辑)
# ==========================================================
# 这是一个专门用来做“阅读理解”的 Prompt
summary_prompt = ChatPromptTemplate.from_template(
    """
    请将以下的对话内容总结为一个简短的摘要（Summary）。
    摘要应包含关键信息，去除冗余的寒暄。
    
    对话内容：
    {chat_history}
    
    摘要：
    """
)

# 格式化Prompt -> 调用LLM -> 解析成字符串
summarize_chain = summary_prompt | llm | StrOutputParser()

# ==========================================================
# 4. 执行总结
# ==========================================================
# history.messages 转换成字符串传给 chain
# (注：如果模型支持直接读 Message 对象，也可以用 MessagesPlaceholder)
chat_history_str = "\n".join([f"{msg.type}: {msg.content}" for msg in history.messages])

summary = summarize_chain.invoke({"chat_history": chat_history_str})

print("\n--- 优化后：生成的摘要 ---")
print(summary)

--- 原始对话条数: 6 ---

--- 优化后：生成的摘要 ---
人类询问AI的身份，AI介绍自己为无所不能的AI助手小智。


举例2：如果SummaryMemory前，已经有历史消息，可以调用from_messages()实例化

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser


# 1. 创建大模型
llm = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

# =========================================================
# 2. 准备历史消息 (替代 ChatMessageHistory)
# =========================================================
# 使用 Core 组件 InMemoryChatMessageHistory
history = InMemoryChatMessageHistory()

# 假设的原始消息
history.add_user_message("你好，你是谁？")
history.add_ai_message("我是AI助手小智")

print(f"--- 初始历史消息数: {len(history.messages)} ---")

# =========================================================
# 3. 定义总结链
# =========================================================
# 可以清楚地定义如何做总结，而不是依赖内部写死的英文 Prompt
summary_prompt = ChatPromptTemplate.from_template(
    """
    请阅读以下的对话历史，并将其压缩为一个简练的摘要（Summary）。
    摘要应包含关键信息，如用户身份、AI身份等。

    对话历史：
    {history_str}

    摘要：
    """
)

# 构建 LCEL 链：格式化 -> LLM -> 解析字符串
summarizer_chain = summary_prompt | llm | StrOutputParser()

# 辅助函数：将消息对象转为字符串，并生成摘要
def get_current_summary(chat_history):
    # 将 Message 对象列表转换为文本字符串
    history_str = "\n".join([f"{msg.type}: {msg.content}" for msg in chat_history.messages])
    # 调用链生成摘要
    return summarizer_chain.invoke({"history_str": history_str})

# =========================================================
# 4. 操作流程演示
# =========================================================

# A. 打印初始摘要
print("\n[操作 1] 生成初始摘要...")
summary_v1 = get_current_summary(history)
print(f"摘要内容: {summary_v1}")

# B. 添加新对话
print("\n[操作 2] 添加新对话...")
history.add_user_message("我的名字叫小明")
history.add_ai_message("很高兴认识你")

# C. 再次生成摘要
print("\n[操作 3] 生成更新后的摘要...")
summary_v2 = get_current_summary(history)
print(f"摘要内容: {summary_v2}")

# D. 查看原始消息
print("\n[操作 4] 查看完整历史记录...")
print(history.messages)

--- 初始历史消息数: 2 ---

[操作 1] 生成初始摘要...
摘要内容: 用户询问AI的身份，AI回应称自己是AI助手小智。

[操作 2] 添加新对话...

[操作 3] 生成更新后的摘要...
摘要内容: 用户小明向AI助手小智问好并介绍自己，小智回应并表达欢迎。对话中包含用户身份为小明，AI身份为小智。

[操作 4] 查看完整历史记录...
[HumanMessage(content='你好，你是谁？', additional_kwargs={}, response_metadata={}), AIMessage(content='我是AI助手小智', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='我的名字叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]


# 3、ConversationSummaryBufferMemory的使用

举例1：

In [3]:
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")

# 1. 定义模型
llm = ChatOpenAI(
    model=os.getenv("LLM_MODEL_ID")
)

# 2. 定义图 (WorkFlow)
# MessagesState 内置了 message 列表，会自动 append 新消息
workflow = StateGraph(state_schema=MessagesState)

# 定义一个节点：直接调用模型
def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# 3. 核心：加上 MemorySaver (Checkpointer)
memory = MemorySaver()

# 4. 编译成应用
app = workflow.compile(checkpointer=memory)

# 5. 调用 (自动管理历史)
config = {"configurable": {"thread_id": "user_1"}} # thread_id 相当于 session_id

# 第一轮
print("--- Round 1 ---")
for chunk in app.stream({"messages": [HumanMessage(content="我是小明")]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

# 第二轮 
print("\n--- Round 2 ---")
for chunk in app.stream({"messages": [HumanMessage(content="我叫什么？")]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

--- Round 1 ---
================================ Human Message =================================

我是小明
================================== Ai Message ==================================

你好，小明！很高兴见到你。有什么我可以帮你的吗？是想聊聊天，还是有具体的问题需要解答？😊

--- Round 2 ---
================================ Human Message =================================

我叫什么？
================================== Ai Message ==================================

你叫小明！😊 是的，我之前已经知道你的名字了。有什么我可以帮你的吗？


对比组：


In [5]:
import os
import dotenv
from typing import Literal
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver

dotenv.load_dotenv()

# 1. 定义模型
llm = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

# =================================================================
# 2. 定义状态 (State Schema)
# =================================================================
class AgentState(MessagesState):
    summary: str

# =================================================================
# 3. 定义逻辑节点 (Nodes)
# =================================================================
def call_model(state: AgentState):
    """主对话节点：负责调用大模型"""
    summary = state.get("summary", "")
    messages = state["messages"]

    # --- 逻辑 A: 注入长期记忆 ---
    # 如果有摘要，将其作为 SystemMessage 插入上下文的最前端
    if summary:
        system_message = SystemMessage(content=f"【长期记忆摘要】\n{summary}")
        # 注意：这里我们构造一个临时的消息列表传给 LLM，不修改 state 里的实际存储
        messages = [system_message] + messages

    response = llm.invoke(messages)
    return {"messages": [response]}


def summarize_conversation(state: AgentState):
    """记忆整理节点：负责将旧对话压缩为摘要 (Buffer -> Summary)"""
    
    # --- 逻辑 B: 缓冲区控制 (Buffer Logic) ---
    stored_messages = state["messages"]
    
    if len(stored_messages) <= 2:
        return {} # 没达到阈值，跳过
    
    # 1. 切分：找出需要被“压缩”的旧消息
    to_summarize = stored_messages[:-2]
    
    # 2. 生成摘要：基于“旧摘要” + “待压缩消息”生成“新摘要”
    current_summary = state.get("summary", "")
    summary_prompt = ChatPromptTemplate.from_template(
        "当前长期记忆：{summary}\n\n新增对话片段：\n{new_lines}\n\n请将新增对话合并入长期记忆，生成一段更新后的精炼摘要："
    )
    chain = summary_prompt | llm | StrOutputParser()
    
    conversation_str = "\n".join([f"{m.type}: {m.content}" for m in to_summarize])
    new_summary = chain.invoke({"summary": current_summary, "new_lines": conversation_str})
    
    # 3. 物理删除：关键步骤
    delete_messages = [RemoveMessage(id=m.id) for m in to_summarize]
    
    return {"summary": new_summary, "messages": delete_messages}

def should_continue(state: AgentState) -> str:
    """边逻辑：决定是否触发记忆整理"""
    messages = state["messages"]
    
    # 阈值判断：如果原文超过 4 条，就触发总结
    if len(messages) > 4:
        return "summarize_conversation"
    return END

# =================================================================
# 4. 构建图 (Graph Construction)
# =================================================================
workflow = StateGraph(AgentState)

# 添加节点
workflow.add_node("conversation", call_model)
workflow.add_node("summarize_conversation", summarize_conversation)

# 定义流向
# 开始 -> 对话 -> 判断(是否太长?) -> 总结 -> 结束
#                   |
#                   V
#                  结束
workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

# 编译应用
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# =================================================================
# 5. 运行演示
# =================================================================
# 只要 thread_id 不变，记忆就会一直存在（并自动被压缩）
config = {"configurable": {"thread_id": "v1_user_test"}}

print("--- Round 1: 自我介绍 ---")
# 此时 buffer 为空，直接回复
input1 = "你好，我是小明。我是一名控制工程的研究生。"
resp1 = app.invoke({"messages": [HumanMessage(content=input1)]}, config)
print(f"AI: {resp1['messages'][-1].content}")

print("\n--- Round 2: 聊聊研究方向 ---")
# 此时 buffer 有 2 条，未触发总结
input2 = "我主要研究多模态水下声学目标识别。"
resp2 = app.invoke({"messages": [HumanMessage(content=input2)]}, config)
print(f"AI: {resp2['messages'][-1].content}")

print("\n--- Round 3: 触发总结 (Buffer Overflow) ---")
# 此时 buffer 将达到 6 条 > 阈值 4。
# 系统将自动执行 summarize_conversation：
# 1. 把 Round 1 (身份) 和 Round 2 (研究方向) 压缩进 summary
# 2. 删除 Round 1 和 Round 2 的原始消息
# 3. 只保留 Round 3 的原文
input3 = "在这个领域，Transformer模型有什么应用？"
resp3 = app.invoke({"messages": [HumanMessage(content=input3)]}, config)
print(f"AI: {resp3['messages'][-1].content}")

# =================================================================
# 6. 验证状态
# =================================================================
print("\n========== 状态透视 ==========")
snapshot = app.get_state(config)
current_summary = snapshot.values.get("summary")
current_messages = snapshot.values.get("messages")

print(f"【长期记忆 (Summary)】: \n{current_summary}")
print(f"【短期显存 (Buffer)】: {[m.content for m in current_messages]}")
print(f"【验证】: 原始的小明身份信息是否已从 Buffer 中删除？ -> {'我是小明' not in str(current_messages)}")

--- Round 1: 自我介绍 ---
AI: 你好，小明！很高兴认识你。你是控制工程的研究生，这说明你正在学习一个非常有趣且应用广泛的领域。控制工程涉及系统的设计、分析和优化，广泛应用于工业自动化、机器人技术、航空航天、汽车工程等多个领域。

你目前的研究方向是？是控制系统设计、建模与仿真，还是控制算法开发，或者是某个具体应用方向，比如智能控制、鲁棒控制、自适应控制、或者控制理论在新兴技术中的应用？如果你愿意分享，我很乐意和你一起讨论相关课题，或者帮你解答一些技术问题。😊

--- Round 2: 聊聊研究方向 ---
AI: 你好，小明！很高兴得知你研究的是**多模态水下声学目标识别**这个方向，这是一个非常前沿且具有挑战性的领域。水下声学目标识别在海洋探测、水下机器人、声呐系统、军事水下监控等应用中都非常重要。而“多模态”意味着你可能同时融合了**声学数据**（比如声呐回波）和其他类型的传感器数据（如光学、惯性导航、压力传感器等），这大大增强了目标识别的准确性和鲁棒性。

在你研究过程中，可能涉及以下几个方面：

---

### 📘 1. **多模态数据融合方法**
- 你可能会用到**多传感器数据融合技术**（Multiple Sensor Data Fusion），例如贝叶斯网络、卡尔曼滤波、深度学习的多模态融合方法（如CNN+RNN、Transformer、Cross-Attention等）。
- 是否考虑了**时空对齐问题**？比如在水下环境中，不同传感器的时间和空间同步是一个关键挑战。

---

### 🧠 2. **数据预处理与特征提取**
- 在水下声学信号处理中，可能会有**噪声干扰**（如海洋环境噪声、多路径效应等），你可能需要做一些**降噪、滤波、去噪**的预处理。
- 特征提取方面，常用的方法包括：
  - **时频分析**（如STFT、小波变换、CWT）
  - **深度学习中的自动特征提取**（如使用CNN、Transformer来提取声学信号或图像的高级特征）
  - 对于多模态数据，你可能需要设计融合不同模态的特征表示方式。

---

### 🤖 3. **深度学习与目标识别模型**
- 在当前研究中，**卷积神经网络（CNN）**、**循环神经网络（RNN）**、**Transformer**、**自注意力机制**、

举例2：模拟客服交互

In [6]:
import os
import dotenv
from typing import Literal

# 1. 导入核心组件
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 2. 导入 LangGraph 组件 (必须安装 langgraph)
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver

# 加载环境变量
dotenv.load_dotenv()

# =========================================================
# 3. 配置模型与状态
# =========================================================
llm = ChatOpenAI(
    model=os.getenv("LLM_MODEL_ID"), 
)

# 定义状态：继承默认的 MessagesState，并额外增加 summary 字段
class AgentState(MessagesState):
    summary: str

# =========================================================
# 4. 定义逻辑节点 (Nodes)
# =========================================================

def call_model(state: AgentState):
    """主对话节点：负责生成回复"""
    
    # A. 获取状态
    summary = state.get("summary", "")
    messages = state["messages"]
    
    # B. 构造 System Prompt (包含人设 + 长期记忆摘要)
    # 这是电商客服的人设
    persona = "你是电商客服助手，用中文友好回复用户问题。保持专业但亲切的语气。"
    
    if summary:
        system_msg_content = f"{persona}\n\n【之前的对话摘要】: {summary}"
    else:
        system_msg_content = persona
        
    # C. 构造临时消息列表传给 LLM (System + History)
    # 注意：不修改 state["messages"]，只是临时组合发给模型
    messages_for_llm = [SystemMessage(content=system_msg_content)] + messages
    
    response = llm.invoke(messages_for_llm)
    return {"messages": [response]}


def summarize_conversation(state: AgentState):
    """记忆整理节点：核心优化点 (Buffer -> Summary)"""
    
    stored_messages = state["messages"]
    
    # 阈值控制：如果消息少于等于 4 条 (2轮)，就不总结
    if len(stored_messages) <= 4:
        return {}
    
    # 1. 切分：保留最后 2 条原文，前面的全部去总结
    to_summarize = stored_messages[:-2]
    
    # 2. 生成摘要
    current_summary = state.get("summary", "")
    prompt = ChatPromptTemplate.from_template(
        "当前摘要: {summary}\n\n新增对话:\n{new_lines}\n\n请将新增对话合并到摘要中，保留关键信息(如订单号、意图):"
    )
    chain = prompt | llm | StrOutputParser()
    
    conversation_str = "\n".join([f"{m.type}: {m.content}" for m in to_summarize])
    new_summary = chain.invoke({"summary": current_summary, "new_lines": conversation_str})
    
    # 3. 物理删除：从状态中移除旧消息，释放 Context Window
    delete_ops = [RemoveMessage(id=m.id) for m in to_summarize]
    
    return {"summary": new_summary, "messages": delete_ops}


def should_continue(state: AgentState) -> str:
    """边逻辑：决定是否触发总结"""
    messages = state["messages"]
    # 如果消息超过 4 条，就去总结
    if len(messages) > 4:
        return "summarize_conversation"
    return END

# =========================================================
# 5. 构建图 (Graph)
# =========================================================
workflow = StateGraph(AgentState)

workflow.add_node("conversation", call_model)
workflow.add_node("summarize_conversation", summarize_conversation)

workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

# 启用持久化记忆
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# =========================================================
# 6. 执行对话 (模拟用户输入)
# =========================================================
dialogue_inputs = [
    "你好，我想查询订单12345的状态",
    "这个订单是上周五下的",
    "我现在急着用，能加急处理吗",
    "等等，我可能记错订单号了，应该是12346",
    "对了，你们退货政策是怎样的"
]

config = {"configurable": {"thread_id": "customer_123"}}

print(f"=== 开始对话 (Thread ID: {config['configurable']['thread_id']}) ===\n")

for i, user_input in enumerate(dialogue_inputs):
    print(f"--- Round {i+1} ---")
    print(f"用户: {user_input}")
    
    # 调用图
    events = app.stream(
        {"messages": [HumanMessage(content=user_input)]}, 
        config, 
        stream_mode="values"
    )
    
    # 获取最后一次输出
    last_response = None
    for event in events:
        if "messages" in event:
            last_response = event["messages"][-1]
            
    print(f"客服: {last_response.content}\n")

# =========================================================
# 7. 查看当前记忆状态
# =========================================================
print("\n=== 最终记忆透视 ===")
snapshot = app.get_state(config)
summary = snapshot.values.get("summary")
buffer_msgs = snapshot.values.get("messages")

print(f"【长期摘要 (Summary)】:\n{summary}")
print(f"\n【短期原文 (Buffer)】 ({len(buffer_msgs)}条):")
for m in buffer_msgs:
    print(f"  [{m.type}]: {m.content}")

=== 开始对话 (Thread ID: customer_123) ===

--- Round 1 ---
用户: 你好，我想查询订单12345的状态
客服: 您好！很高兴为您服务。请问订单号12345的订单是通过哪个平台或店铺下单的呢？这样我可以更准确地帮您查询订单状态。

--- Round 2 ---
用户: 这个订单是上周五下的
客服: 您好，感谢您的补充！订单12345是上周五下的，我这就为您查询最新的订单状态。请您稍等片刻，我尽快为您反馈相关信息。如果需要其他帮助，也欢迎随时告诉我！

--- Round 3 ---
用户: 我现在急着用，能加急处理吗
客服: 您好，理解您的急迫心情！我们非常重视您的需求，订单12345已经为您标记为加急处理，会优先安排物流和配送。请您保持电话畅通或关注短信通知，以便我们及时与您联系。如果还有其他问题，欢迎随时告知！祝您一切顺利！

--- Round 4 ---
用户: 等等，我可能记错订单号了，应该是12346
客服: 您好，感谢您的提醒！您可能记错了订单号，我们刚刚收到新的订单号12346的信息。为了确保准确无误，请您确认一下订单号是否正确，或者提供一些订单的其他信息（如购买时间、商品名称等），我们可以帮您进一步核实。再次感谢您的理解与配合！

--- Round 5 ---
用户: 对了，你们退货政策是怎样的
客服: 您好，感谢您的提问！我们的退货政策如下：

如果您对商品不满意，可以在签收后 **7天内** 申请无理由退货（部分特殊商品除外，如定制类、易损耗品等，具体以商品页面说明为准）。退货时请保留商品的原始包装和发票，确保商品完好无损。

申请退货的流程是：  
1. 登录您的账户，进入“我的订单”页面；  
2. 找到对应订单，点击“申请退货”；  
3. 填写退货原因并提交；  
4. 我们会在**24小时内**审核并通知您退货结果。

如果需要帮助，我们也可以协助您完成退货操作哦！祝您购物愉快！


=== 最终记忆透视 ===
【长期摘要 (Summary)】:
当前摘要：  
用户询问订单状态，最初提到订单号12345，AI确认该订单的下单时间为上周五，并正在查询其最新状态，请求用户稍等。随后用户表示可能记错了订单号，应为12346，AI确认已收到新订单号12346的信息，并建议用户确认订单号